# 9.4 향후 판매량 예측 경진대회 모델 성능 개선
- [향후 판매량 예측 경진대회 링크](https://www.kaggle.com/c/competitive-data-science-predict-future-sales)
- [모델링 코드 참고 링크](https://www.kaggle.com/dkomyagin/predict-future-sales-lightgbm-framework)

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore')

data_path = '/kaggle/input/competitive-data-science-predict-future-sales/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 9.4.1 피처 엔지니어링1: 피처명 한글화와 데이터 다운캐스팅

In [ ]:
sales_train = sales_train.rename(columns={'date':'날짜',
                                          'date_block_num':'월ID',
                                          'shop_id':'상점ID',
                                          'item_id':'상품ID',
                                          'item_price':'판매가',
                                          'item_cnt_day':'판매량'})

shops = shops.rename(columns={'shop_name':'상점명',
                              'shop_id':'상점ID'})

items = items.rename(columns={'item_name':'상품명',
                              'item_id':'상품ID',
                              'item_category_id':'상품분류ID'})

item_categories = item_categories.rename(columns={'item_category_name':'상품분류명',
                                                  'item_category_id':'상품분류ID'})

test = test.rename(columns={'shop_id':'상점ID',
                            'item_id':'상품ID'})

In [ ]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem) / start_mem))
        
    return df

all_df = [sales_train, shops, items, item_categories, test]

for df in all_df:
    df = downcast(df)

### 9.4.2 피처 엔지니어링2: 개별 데이터 피처 엔지니어링

* sales_train 이상치 제거 및 전처리

In [ ]:
sales_train = sales_train[sales_train['판매가'] > 0]
sales_train = sales_train[sales_train['판매가'] < 50000]

sales_train = sales_train[sales_train['판매량'] > 0]
sales_train = sales_train[sales_train['판매량'] < 1000]

In [ ]:
print(shops['상점명'][0], '||', shops['상점명'][57])
print(shops['상점명'][1], '||', shops['상점명'][58])
print(shops['상점명'][10], '||', shops['상점명'][11])
print(shops['상점명'][39], '||', shops['상점명'][40])

In [ ]:
# sales_train 데이터에서 상점ID 수정
sales_train.loc[sales_train['상점ID'] == 0, '상점ID'] = 57
sales_train.loc[sales_train['상점ID'] == 1, '상점ID'] = 58
sales_train.loc[sales_train['상점ID'] == 10, '상점ID'] = 11
sales_train.loc[sales_train['상점ID'] == 39, '상점ID'] = 40

# test 데이터에서 상점 ID 수정
test.loc[test['상점ID'] == 0, '상점ID'] = 57
test.loc[test['상점ID'] == 1, '상점ID'] = 58
test.loc[test['상점ID'] == 10, '상점ID'] = 11
test.loc[test['상점ID'] == 39, '상점ID'] = 40

* shops 파생 피처 생성 및 인코딩

In [ ]:
shops['도시'] = shops['상점명'].apply(lambda x: x.split()[0])

In [ ]:
shops['도시'].unique()

In [ ]:
shops.loc[shops['도시'] == '!Якутск', '도시'] = 'Якутск'

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
shops['도시'] = label_encoder.fit_transform(shops['도시'])

In [ ]:
shops = shops.drop('상점명', axis=1)
shops.head()

* items 파생 피처 생성

In [ ]:
items = items.drop(['상품명'], axis=1)

In [ ]:
items['첫 판매월'] = sales_train.groupby('상품ID').agg({'월ID':'min'})['월ID']
items.head()

In [ ]:
items[items['첫 판매월'].isna()]

In [ ]:
items['첫 판매월'] = items['첫 판매월'].fillna(34)

* item_categories 파생 피처 생성 및 인코딩

In [ ]:
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])

In [ ]:
item_categories['대분류'].value_counts()

In [ ]:
def make_etc(x):
    if len(item_categories[item_categories['대분류'] == x]) >= 5:
        return x
    else:
        return 'etc'

item_categories['대분류'] = item_categories['대분류'].apply(make_etc)

In [ ]:
item_categories.head()

In [ ]:
label_encoder = LabelEncoder()

item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])

item_categories = item_categories.drop('상품분류명', axis=1)

### 9.4.3 피처 엔지니어링3: 데이터 조합 및 파생 피처 생성

* 데이터 조합

In [ ]:
from itertools import product

train = []
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID'] == i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID'] == i, '상품ID'].unique()
    
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID', '상점ID', '상품ID']
train = pd.DataFrame(np.vstack(train), columns=idx_features)

* 파생 피처 생성

In [ ]:
group = sales_train.groupby(idx_features).agg({'판매량':'sum', '판매가':'mean'})

group = group.reset_index()
group = group.rename(columns={'판매량':'월간 판매량', '판매가':'평균 판매가'})

train = train.merge(group, on=idx_features, how='left')

train.head()

In [ ]:
import gc

del group
gc.collect()

In [ ]:
# 상품 판매건수 피처 추가
group = sales_train.groupby(idx_features).agg({'판매량':'count'})
group = group.reset_index()
group = group.rename(columns={'판매량':'판매건수'})

train = train.merge(group, on=idx_features, how='left')

del group, sales_train
gc.collect()

train.head()

### 9.4.4 피처엔지니어링4: 데이터 합치기

* 테스트 데이터 이어붙이기

In [ ]:
test['월ID'] = 34

all_data = pd.concat([train, test.drop('ID', axis=1)], ignore_index=True, keys=idx_features)
all_data = all_data.fillna(0)
all_data.head()

* 모든 데이터 병합

In [ ]:
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

all_data = downcast(all_data)

In [ ]:
del shops, items, item_categories
gc.collect()

### 9.4.5 피처 엔지니어링5: 시차 피처 생성

* 기준 피처별 월간 평균 판매량 파생 피처 생성

In [ ]:
def add_mean_features(df, mean_features, idx_features):
    # 기준 피처 확인
    assert (idx_features[0] == '월ID') and len(idx_features) in [2, 3]
    
    if len(idx_features) == 2:
        feature_name = idx_features[1] + '별 평균 판매량'
    else:
        feature_name = idx_features[1] + ' ' + idx_features[2] + '별 평균 판매량'
    
    group = df.groupby(idx_features).agg({'월간 판매량':'mean'})
    group = group.reset_index()
    group = group.rename(columns = {'월간 판매량':feature_name})
    
    df = df.merge(group, on=idx_features, how='left')
    df = downcast(df, verbose=False)
    
    mean_features.append(feature_name)
    
    del group
    gc.collect()
    
    return df, mean_features

In [ ]:
item_mean_features = []

all_data, item_mean_features = add_mean_features(df=all_data, mean_features=item_mean_features, idx_features=['월ID', '상품ID'])

all_data, item_mean_features = add_mean_features(df=all_data, mean_features=item_mean_features, idx_features=['월ID', '상품ID', '도시'])

In [ ]:
item_mean_features

In [ ]:
shop_mean_features = []

all_data, shop_mean_features = add_mean_features(df=all_data, mean_features=shop_mean_features, idx_features=['월ID', '상점ID', '상품분류ID'])

In [ ]:
shop_mean_features

* 시차 피처 생성 원리 및 함수 구현

In [ ]:
def add_lag_features(df, lag_features_to_clip, idx_features, lag_feature, nlags=3, clip=False):
    df_temp = df[idx_features + [lag_feature]].copy()
    
    for i in range(1, nlags + 1):
        lag_feature_name = lag_feature + '_시차' + str(i)
        
        df_temp.columns = idx_features + [lag_feature_name]
        df_temp['월ID'] += i
        df = df.merge(df_temp.drop_duplicates(), on=idx_features, how='left')
        df[lag_feature_name] = df[lag_feature_name].fillna(0)
        
        if clip:
            lag_features_to_clip.append(lag_feature_name)
        
    df = downcast(df, False)
    
    del df_temp
    gc.collect()
    
    return df, lag_features_to_clip

* 시차 피처 생성1: 월간 판매량

In [ ]:
lag_features_to_clip = []
idx_features = ['월ID', '상점ID', '상품ID']
all_data, lag_features_to_clip = add_lag_features(df = all_data, lag_features_to_clip=lag_features_to_clip, idx_features=idx_features, 
                                                  lag_feature='월간 판매량', nlags=3, clip=True)

In [ ]:
all_data.head().T

In [ ]:
lag_features_to_clip

* 시차 피처 생성2: 판매건수, 평균 판매가

In [ ]:
all_data, lag_features_to_clip = add_lag_features(df=all_data, lag_features_to_clip=lag_features_to_clip, idx_features=idx_features,
                                                 lag_feature='판매건수', nlags=3)

all_data, lag_features_to_clip = add_lag_features(df=all_data, lag_features_to_clip=lag_features_to_clip, idx_features=idx_features, lag_feature='평균 판매가', nlags=3)

* 시차 피처 생성3: 평균 판매량

In [ ]:
for item_mean_feature in item_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data, lag_features_to_clip=lag_features_to_clip, idx_features=idx_features,
                                                      lag_feature=item_mean_feature, nlags=3, clip=True)

    
all_data = all_data.drop(item_mean_features, axis=1)

In [ ]:
for shop_mean_feature in shop_mean_features:
    all_data, lag_features_to_clip = add_lag_features(df=all_data, lag_features_to_clip=lag_features_to_clip, idx_features=['월ID', '상점ID', '상품분류ID'],
                                                     lag_feature=shop_mean_feature, nlags=3, clip=True)
    
all_data = all_data.drop(shop_mean_features, axis=1)

In [ ]:
all_data = all_data.drop(all_data[all_data['월ID'] < 3].index)

### 9.4.6 피처엔지니어링6: 기타 피처 엔지니어링

* 월간 판매량 시차 피처들의 평균

In [ ]:
all_data['월간 판매량 시차평균'] = all_data[['월간 판매량_시차1', '월간 판매량_시차2', '월간 판매량_시차3']].mean(axis=1)

In [ ]:
all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']] = all_data[lag_features_to_clip + ['월간 판매량', '월간 판매량 시차평균']].clip(0, 20)

In [ ]:
all_data['시차변화량1'] = all_data['월간 판매량_시차1'] / all_data['월간 판매량_시차2']
all_data['시차변화량1'] = all_data['시차변화량1'].replace([np.inf, -np.inf], np.nan).fillna(0)

all_data['시차변화량2'] = all_data['월간 판매량_시차2'] / all_data['월간 판매량_시차3']
all_data['시차변화량2'] = all_data['시차변화량2'].replace([np.inf, -np.inf], np.nan).fillna(0)

* 신상 여부

In [ ]:
all_data['신상여부'] = all_data['첫 판매월'] == all_data['월ID']

* 첫 판매 후 경과 기간

In [ ]:
all_data['첫 판매 후 기간'] = all_data['월ID'] - all_data['첫 판매월']

In [ ]:
all_data['월'] = all_data['월ID'] % 12

In [ ]:
all_data = all_data.drop(['첫 판매월', '평균 판매가', '판매건수'], axis=1)

all_data = downcast(all_data, False)

### 9.4.7 피처 엔지니어링7: 마무리

In [ ]:
all_data.info()

In [ ]:
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']


del all_data
gc.collect()

### 9.4.8 모델 훈련 및 성능 검증

In [ ]:
import lightgbm as lgb

params = {
    'metric': 'rmse',
    'num_leaves': 255,
    'learning_rate': 0.005,
    'feature_fraction': 0.75,
    'bagging_fraction':0.75,
    'force_col_wise': True,
    'random_state': 10
}

cat_features = ['상점ID', '도시', '상품분류ID', '대분류', '월']

dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

lgb_model = lgb.train(params=params, train_set=dtrain, num_boost_round=1500, valid_sets=(dtrain, dvalid), early_stopping_rounds=150,
                     categorical_feature=cat_features, verbose_eval=100)

### 9.4.9 예측 및 결과 제출

In [ ]:
preds = lgb_model.predict(X_test).clip(0, 20)

submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)

In [ ]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect()

### 핵심 요약
* **훈련 데이터가 여러 파일**로 제공되면 공통 피처를 기준으로 병합해 사용한다.
* 직접적인 **타깃값이 제공되지 않기도**한다. 이럴때는 존재하는 피처를 조합하거나 계산하여 타깃값을 구한다.
* 회귀 문제에서는 특정 피처를 기준으로 데이터를 **그룹화**해 값을 **집계**해 사용하는 일이 많다. 집계 방법은 합, 평균, 중간값, 표준편차, 분산, 개수, 최솟값, 최댓값  등이 있다.
* 피처가 다양할 때는 **피처명을 한글화**하는 것도 좋은 방법이다.
* 데이터가 크면 메모리 관리도 신경 써야 한다.
    * **데이터 다운캐스팅**은 작은 데이터 타입으로 변환하는 작업을 말한다.
    * **가비지 컬렉션**은 더는 사용하지 않는 영역을 해제하는 기능이다.
* **이상치**가 있을 때는 해당 데이터 자체를 제거하거나 적절한 값으로 바꿔준다.
* 둘 이상의 피처를 **조합**하면 유용한 데이터의 수가 늘어나는 효과가 있다.
* 분류 피처의 각 분류별 데이터 수가 적다면 **대분류**로 다시 묶어 훈련하는 것도 좋은 방법이다.
* **시계열 데이터**에서는 시간 흐름 자체가 중요한 정보이다. OOF 예측이나 데이터를 무작위로 섞는 등 시간 순서를 무시하는 기법은 이요할 수 없다.
* **시차피처**란 과거 시점에 관한 피처로, 성능 향상에 도움되는 경우가 많아서 시계열 문제에서 자주 만드는 파생 피처이다.